In [1]:
import requests
import time
from datetime import timedelta
import pickle
from gensim.models import doc2vec
import numpy as np
import time 

from sklearn.cluster import DBSCAN

from crashsimilarity.downloader import SocorroDownloader
from crashsimilarity import utils
from crashsimilarity.distance import DistanceCalculator

In [2]:
signature2 = 'OOM | large | js::AutoEnterOOMUnsafeRegion::crash | js::AutoEnterOOMUnsafeRegion::crash | js::TenuringTracer::moveToTenured'

In [3]:
def download_stack_traces(sigs, traces_num=1):
    from_date = utils.utc_today() - timedelta(days=360)
    if isinstance(sigs, str):
        sigs = [sigs]
    params = {'signature': ['^'+i for i in sigs],
              'date': ['>=' + str(from_date)],
              '_facets': ['proto_signature'],
              '_facets_size': traces_num
             }
    _SUPER_SEARCH_URL = 'https://crash-stats.mozilla.com/api/SuperSearch'
    r = requests.get(_SUPER_SEARCH_URL, params)
    return r.json()['facets']['proto_signature']

In [4]:
traces2 = download_stack_traces(signature2, traces_num=1000)

In [5]:
traces2 = set([i['term'] for i in traces2])
corpus = []
for x in traces2:
    words = utils.StackTraceProcessor.preprocess(x)
    if words[0] == '@0x':
        words = words[1:]
    corpus.append(words)
len(corpus)

1000